In [1]:
import numpy as np
from chapter1 import *

### **1. 변수 (Variable)**

In [2]:
data = np.array(1.0)
x = Variable(data)      # x는 Variable의 인스턴스

print(f"val: {x.data}, dim: {x.data.ndim}")

val: 1.0, dim: 0


### **2. 함수 (Function)**

In [3]:
x = Variable(np.array(10))
f = Square()    # Function 클래스를 상속
y = f(x)        # 입력으로 사용되는 x는 Variable 인스턴스

print(f"val: {y.data}, type: {type(y)}")

val: 100, type: <class 'chapter1.Variable'>


### 3. **함수 연결 (Composite Function)**

In [7]:
sqr = Square()
exp = Exp()

# 합성 함수를 통해 "계산 그래프" 생성
x = Variable(np.array(0.5))
h1 = sqr(x)
h2 = exp(h1)
y = sqr(h2)

print(f"val: {y.data: .4f}, type: {type(y)}")

val:  1.6487, type: <class 'chapter1.Variable'>


### **4. 수치 미분 (Numerical Differential)**

In [6]:
def f(x):
    sqr = Square()
    exp = Exp()

    return sqr(exp(sqr(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x) # 중앙 차분을 이용한 수치 미분
print(dy)

 0.000000032974
